In [178]:
import networkx as nx
import json

# --- Load POS tags with attributes ---
with open("pos_tags.json", "r", encoding="utf-8") as f:
    pos_tags = json.load(f)


# --- Create graph ---
G = nx.Graph()

# --- Load edge list ---
G = nx.read_edgelist("cooccurrence_pairs_window2.txt", delimiter="\t")

# --- Add node attributes from pos_tags ---
for node in G.nodes():
    if node in pos_tags:
        G.nodes[node]["pos"] = pos_tags[node].get("pos")
        G.nodes[node]["gender"] = pos_tags[node].get("gender")
    else:
        G.nodes[node]["pos"] = None
        G.nodes[node]["gender"] = None



In [179]:
G.number_of_edges()
G.number_of_nodes()

15732

In [180]:
degree_centrality = nx.degree_centrality(G)

# Get node with maximum centrality
max_degree_node = max(degree_centrality, key=degree_centrality.get)

node_name = max_degree_node  
node_attrs = G.nodes[max_degree_node]

print(f"Node with highest degree centrality: {node_name}")
print(f"Centrality: {degree_centrality[max_degree_node]}")
print(f"Attributes: {node_attrs}")


Node with highest degree centrality: find
Centrality: 0.06414086834911957
Attributes: {'pos': 'VERB', 'gender': None}
